*이름*: 이장범

학번:20171722

# Neural networks with PyTorch

Pytorch의 `nn.module`을 활용하여 만드는 유용한 방법을 학습합니다.

<div style="text-align:center"><img src='https://drive.google.com/uc?export=download&id=1J2SeiPpVJs1-ML2BdLrcxkGGmHpRxIVE' width="250" height="200"> 

### Lego block coding! </div>

In [5]:
import torch
import numpy as np

from torch import nn
import torch.nn.functional as F
from collections import OrderedDict

torch.manual_seed(0)

*이탤릭체 텍스트*`nn.Linear`: $Z^{[\ell]} = A^{[\ell-1]}W^T+b$
연산.

해당 layer의 

- 입력 차원 `n_input=30`
- 출력 차원 `n_output=60`


In [6]:
# Example of nn.linear
linear_layer1 = nn.Linear(30, 60)

In [7]:
A = torch.randn(60, 30)
linear_layer1(A).shape

torch.Size([60, 60])

How to get the weights and bias of each `nn.Linear`

In [8]:
# Example of weights
linear_layer1.weight.data = torch.ones_like(linear_layer1.weight)

In [31]:
linear_layer1.weight 

Parameter containing:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True)

### NN example

- input units: 20
- hidden layer: 30, 40
- output units: 3
- activation function: ReLU
- output layer: No activation



In [10]:
# Simple NN construction

class FCN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lin1 = nn.Linear(20, 30)
        self.lin2 = nn.Linear(30, 40)
        self.lin3 = nn.Linear(40, 3)
        self.relu = nn.ReLU(True)
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)
        return x
    


In [11]:
Xtrain = torch.randn(60, 20)

model = FCN()
# model(Xtrain)

In [12]:
model

FCN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [13]:
# Example of parameters() in models
# param_iterator = model.parameters()

# for param in param_iterator:
#     print(param)


In [14]:
# nn.Sequential() example

class FCN_seq(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(nn.Linear(20, 30),
                      nn.ReLU(True),
                      nn.Linear(30, 40),
                      nn.ReLU(True),
                      nn.Linear(40, 3)
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [15]:
model_seq = FCN_seq()
Xtrain.shape
model_seq

FCN_seq(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [35]:
model_seq.fc[0]

Linear(in_features=20, out_features=30, bias=True)

In [17]:
class FCN_seq_v2(nn.Module):
    def __init__(self):
        super().__init__()
        

        
        temp = self.fcn_block(20, 30)+self.fcn_block(30, 40)+[nn.Linear(40,1)]
        self.fc = nn.Sequential(*temp)
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [18]:
model_seq_v2 = FCN_seq_v2()
model_seq_v2

FCN_seq_v2(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)

In [38]:
class FCN_final(nn.Module):
    def __init__(self, in_dim, hlayer, out_dim):
        super().__init__()
        
        l_list = self.fcn_block(in_dim, hlayer[0])
        
        for l1, l2 in zip(hlayer[:-1], hlayer[1:]):
            l_list = l_list + self.fcn_block(l1, l2)
        
        l_list = l_list + [nn.Linear(hlayer[-1], out_dim)]
      
        self.fc = nn.Sequential(*l_list)
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        

In [39]:
hlayer = [30, 40]
in_dim = 20
out_dim= 3

myfcn_final = FCN_final(in_dim, hlayer, out_dim)

In [21]:
myfcn_final

FCN_final(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [22]:
# Ordered dict example
# nn.Sequential() example

class FCN_seq_ordered_dic(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(20, 30)),
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(30, 40)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(40, 3))
                                            ])
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [23]:
# ModuleList(), ModuleDict()

In [ ]:
# state_dict() example
model.state_dict()

# Problem Setup

<div style="text-align:center"> <img src='https://drive.google.com/uc?export=download&id=1FRhniwGeeutBSJQRdW6GzshMfDrPz7oJ' width="250" height="200"> </div>
    
Build a Fully connected neural network with

- 3 layers
- 마지막 layer의 unit 수는 `1` 
  - 마지막 layer의 activation은 없음 (linear layer)
- Data feature 수는 `100`

- input unit 수는 data 크기를 보고 맞추세요
- hidden layer의 unit 수는 `[80, 50]`
  - hidden layer의 activation 함수는 ReLU

- model class 명 `myFCN`
  - instance 명 `my_model` 생성
  - `my_model` 출력

## Problem 1

problem setup에서 구성한 neural network을 `nn.Sequential`을 활용하여 생성하세요

In [40]:
## 사용할 data 
batch_size = 30
num_feature = 100

X_train = torch.randn(batch_size, num_feature)

In [26]:
X_train.shape

torch.Size([30, 100])

In [80]:
from torch.nn.modules.linear import Linear #Linear 불러오기
from torch.nn.modules.activation import ReLU # ReLU 불러오기
# Problem 1 코딩 (매 줄마다 주석 필수 )
class myFCN(nn.Module):
    def __init__(self):
      # 초기화 method
        super().__init__()
        # input data에 num_features는 100, batch_size 는 30
        self.fc = nn.Sequential(nn.Linear(30, 100),
                      nn.ReLU(True), # activation function을 relu로 사용,
                      nn.Linear(100, 80), # hidden layer에 neruon 개수 80이므로, nn.Linear의 2번째 parameter 값을 80으로 설정했다. 
                      nn.ReLU(True), # activation function을 relu로 사용,,
                      nn.Linear(80, 50),# hidden layer에 neruon 개수 80이므로, nn.Linear의 2번째 parameter 값을 80으로 설정했다. 
                      nn.ReLU(True), # activation function을 relu로 사용,,
                      nn.Linear(50,1)# 마지막 출력 layer에 1로 해야하므로, nn.Linear의 3번째 parameter 값을 고정 값 1로 설정했다.
        )
        
    def forward(self, x): # forward 함수 불러오기
        return self.fc(x) # 위에서 만든 self.fc 입력으로 들어오는 x를 넣고, forward pass 진행


In [77]:
my_model=myFCN() # Make myFCN's object 

In [81]:
my_model # object print

myFCN(
  (fc): Sequential(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=100, out_features=80, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=80, out_features=50, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)

## Problem 2

problem setup에서 구성한 neural network을 `OrderedDict`을 활용하여 생성하세요
- 각 layer의 이름을 주고 생성하세요

In [73]:
# 답작성\
class myFCN_dict(nn.Module):
    def __init__(self):
        super().__init__()
        # 초기화 method
        self.fc = nn.Sequential(OrderedDict([('layer1', nn.Linear(30, 100)),
        # input data에 num_features는 100, batch_size 는 30, OrderedDict
                      ('relu1', nn.ReLU(True)),# activation function을 relu로 사용,
                      ('layer2', nn.Linear(100, 80)), # hidden layer에 neruon 개수 80이므로, nn.Linear의 2번째 parameter 값을 80으로 설정했다. 
                      ('relu2',nn.ReLU(True)),# activation function을 relu로 사용,
                      ('layer3', nn.Linear(80, 50)), # hidden layer에 neruon 개수 50이므로, nn.Linear의 2번째 parameter 값을 50으로 설정했다. 
                      ('relu3',nn.ReLU(True)),# activation function을 relu로 사용,
                      ('layer3', nn.Linear(50,1)) # 마지막 출력 layer에 1로 해야하므로, nn.Linear의 3번째 parameter 값을 고정 값 1로 설정했다.
                                            ])
        )
        
    def forward(self, x): # forward 함수 불러오기
        return self.fc(x) # 위에서 만든 self.fc 입력으로 들어오는 x를 넣고, forward pass 진행

In [74]:
my_model2=myFCN_dict() # Make myFCN_dict's object 
my_model2 # object print

myFCN_dict(
  (fc): Sequential(
    (layer1): Linear(in_features=30, out_features=100, bias=True)
    (relu1): ReLU(inplace=True)
    (layer2): Linear(in_features=100, out_features=80, bias=True)
    (relu2): ReLU(inplace=True)
    (layer3): Linear(in_features=50, out_features=1, bias=True)
    (relu3): ReLU(inplace=True)
  )
)